## Segmentation


The goal of segmentation is identify the boundaries between different objects in an image, and to simplify the representation of an image into meaningful boundaries that are easier to analyze. This topic is important because:
- it is a more basic step for the convolutional filters in Neural Networks for extracting image features
- it is the basis for many other image processing tasks such as object detection, object tracking, and image classification

\
Image segmentation can be classified into three categories:
Semantic segmentation
Instance segmentation
Panoptic segmentation

The document will cover the following topics:
- Segmentation as pixel wise classification
    - Probabilistic classification
    - Mixture of Gaussians, EM

- Segmentation as energy minimization
    - Markov Random Fields
    - Energy formulation

- Graph cuts for image segmentaton
    - Basic idea
    - s-t Mincut Algorithm
    - Extension to non-binary case



\
Definition of The Problem\
Identifying groups of pixels in the input which is the image. This is a semantic segmentation, this means all the pixels that belong to a title are grouped together. For example, semantically meaningful groups such as color similarity,




Segmentation Approaches:
- Unsupervised Clustering
    - Grouping what "looks similar"
- Semantic Segmentation
    - Learn a classifier to assign a semantic class $C_k$ to every pixel


## Segmentation as Pixel-Wise Classification

To define the grouping semantics, we use feature space. Grayscale image pixels are classified based on intensity similarity as a 1D representation, while colored images are classified based on color value similarity as a 3D representation.

A filter Bank of 24 filters allow us to operate in 24-dimension. 


\
Basically the method is to apply a threshold, above which is the foreground, and below is the background.


### Probabilistic Classification
- Bayesian Classification
    - Given a measurement $x$, what semantic class $C_k$ should we assign to a pixel?
    - We must recall Bayes Decision Theory in this section (posterior probability):
        - $P(C_k|x) = \frac{p(x|C_k)p(C_k)}{\sum_{j}p(x|C_j)p(C_j)} $
        - where: 
            - $p(x|C_k)$ is the likelihood of the measurement $x$ having been generated by class $C_k$
            -  $p(C_k)$ is the prior probability of class $C_k$
    - In order to build a classifier, we can try either:
        - Discriminative Methods: directly estimating the posterior
        - Generative Methods: estimating likelihood and prior, and then using the Bayes Decision formula


In image recognition, a machine learning model can be taught to recognize objects. Next section will explore how machines recognize classifications by finding patterns. In order to explore the segmentation subject further, we need to take a look at Machine Learning Data Models first to understand the context better.


### Machine Learning Data Models

A machine Learning data model is a program that expresses the relationship between data and finds patterns in the dataset. When an unseen dataset is expressed in a ML data model, the model creates meaningful connections between data, and this can help us make decisions. For example, in natural language processing, machine learning models can parse and correctly recognize the context behind previously unheard sentences or combinations of words.

Machine learning data models can be classified into two types: Generative and Discriminative Models

- A generative model focuses on learning the underlying probability distribution of a given dataset. The fundamental idea behind generative models is to create a model that can generate new data points statistically similar to the original dataset, and new data points are generated from the probability distribution. This model type learns the patterns between data, and creates new realistic networks stemming from one value. This is why, the model focuses on learning the probability distribution that generates data, rather than the classification of data.
Examples:
    - Image or Face generation with Generative adversarial networks (GANs) 
    - Text generation
    - Anomaly detection
    - Gaussian mixture model learning the parameters of the Gaussian mixture that best fits the data \
&nbsp;

- A discriminative model focuses on learning which x-value will map to which y-value. In other words, it learns the direct mapping between input variables and output labels (aka the classification of data) through learned boundaries without considering the underlying probability distribution of the data. The discriminative model learns to find the decision boundary that separates different classes or categories in the input space. This model type can make predictions on previously unseen data based on conditional probability and can be used either for classification or regression problem statements. 
For example: 
    - A convolutional neural network recognizing what an object in an image is
    - A program that predicts the price of a house based on its features
    - Logistic regression program performing sentiment analysis



<p float="left">
  <img src="Segmentation\img\generativedatamodel.jpg" width=45% />
  <img src="Segmentation\img\discriminativedatamodel.jpg" width=45% /> 
</p>

<div style="display: flex;">
        <div style ="padding-left" 10>
            <h3>Generative Model</h3>
            <p>A generative model focuses on learning the underlying probability distribution of a given dataset. The fundamental idea behind generative models is to create a model that can generate new data points statistically similar to the original dataset, and new data points are generated from the probability distribution. This model type learns the patterns between data, and creates new realistic networks stemming from one value. This is why, the model focuses on learning the probability distribution that generates data, rather than the classification of data.
            
Examples:
<ul>
<li>Image or Face generation with Generative adversarial networks (GANs)</li>
<li>Text generation</li>
<li>Anomaly detection</li>
<li>Gaussian mixture model learning the parameters of the Gaussian mixture that best fits the data</li>
</ul>
</p>
        </div>
        <div style ="padding-right" 10>
            <h3>Discriminative Model</h3>
            <p>A discriminative model focuses on learning which x-value will map to which y-value. In other words, it learns the direct mapping between input variables and output labels (aka the classification of data) through learned boundaries without considering the underlying probability distribution of the data. The discriminative model learns to find the decision boundary that separates different classes or categories in the input space. This model type can make predictions on previously unseen data based on conditional probability and can be used either for classification or regression problem statements.
            
Examples:
<ul>
<li>A convolutional neural network recognizing what an object in an image is</li>
<li>A program that predicts the price of a house based on its features</li>
<li>Logistic regression program performing sentiment analysis</li>
<ul>
</p>
        </div>
</div>



### Mixture Model of Gaussian Distribution (MoG)
A mixture model is a statistical model used for representing data that may arise from a mixture of different probability distributions. In simpler terms, it's a way to describe data that might come from several different sources or populations.

For estimating the parameters of a mixture model, we determine component distributions and their corresponding weights, with methods like expectation-maximization (EM) algorithm or Bayesian inference.

Mixture Model of Gaussian Distribution (MoG) is a specific type of mixture model where the data distributions are Gaussian (also known as normal) distributions. MoG offers a flexible data distribution. In a Gaussian mixture model, the assumption is that the observed data is generated by a mixture of several Gaussian distributions with different means and variances. Therefore it is a generative data model.

Mixture Model of Gaussian Distribution will be referred as MoG here on. MoG can be expressed so:
$$ p(x) = \sum_{i=1}^{K} \phi_i \mathcal{N}(x|\mu_i, \Sigma_i) $$

where,
- $p(x)$ is the probability density function of the mixture model
- $\phi_i$ is the mixing coefficient for the $i$-th component
- $\mathcal{N}(x|\mu_i, \Sigma_i)$ represents the Gaussian distribution with mean $\mu_i$ and covariance matrix $\Sigma_i$
- $K$ is the number of components in the mixture

&nbsp;
&nbsp;

### Expectation-Maximization (EM) Algorithm
The Expectation-maximization algorithm (EM), is an iterative method for performing maximum likelihood in certain models.

 - E-Step: assigns samples to mixture model components:
    - $ \pi_j \gamma_j(x_n) $


User Assisted Image Segmentation
- User marks two regions for foreground and background
- Learn a MoG model for the color values in each region
- Use the models to classify all pixels by deciding for the class with the highest posterior probability

M-Step: re-estimates the parameters (separately for each mixture component) based on the soft assignment
$$ \^{N}_j \leftarrow \sum_{n=1}^{N}\gamma_j(x_n) $$

### Pros of MoG, EM
- It provides an interpretation of the task probability functions
- It is a generative model as the values can be generated from the distribution, and it can predict novel data points

#### Cons of MoG, EM
- Local minima
    - k-means is NP-hard (see: computational complexity theory, nondeterministic polynomial time) even with k=2
- Initialization
    - Often a good idea to start with some k-means iteration
- Needs to know number of componens
    - Solution: Model selection
- Needs careful implementation to avoid numerical issues

### Caveats

So far we have explored  bottom-up ways for segmentation, for which we examined individual pixels and neighborhoods to segment an image into regions. Due to the problem for recognition in finding meaningful segments, alternative methods are explored. When the methods in pixel-wise segmentation are applied to real world datasets, they result in very noisy segmentations.

We would like to enforce region constraints with spatial consistency and smooth borders.

For this, in the next section, we will explore pixel neighborhood relations.

### References for this Section

---

[1] v7labs, https://www.v7labs.com/blog/panoptic-segmentation-guide

[2] fiveMinuteStats, EM, https://stephens999.github.io/fiveMinuteStats/intro_to_em.html

[3] fiveMinuteStates, Mixture Models, https://stephens999.github.io/fiveMinuteStats/intro_to_mixture_models.html

[4] neptune.ai, https://neptune.ai/blog/image-segmentation

[5] Mordatch, Igor, "Concept Learning with Energy-Based Models" OpenAI. https://openreview.net/pdf?id=H12Y1dJDG



&nbsp;

## Segmentation as Energy Minimization

In the previous section, we explored semantic pixel based clustering approaches for segmentation. In this section, we will explore Energy Minimization methods.

### Markov Random Fields

Markov Random Fields (MRF) is a method to model a joint distribution of an undirected, connected graph where each node implies a random variable and each edge between nodes is a modeled stochastic dependency.

MRF is an undirected graphical model that explicitly expresses the conditional independence relationship between nodes.

In MRF, we use a class of graphical models to model the conditional probability of a random variable with its given parents.



![](Segmentation/img/mrf.png)

### Conditional Random Fields

We would like to minimize an energy by using neighboring relations, therefore we use a classifier that predicts a label for a single sample without considering "neighbouring" samples.


### Energy Formulation

Joint Probability\
$p(x,y) = \prod_{i} \phi(x_i,y_i) \prod_{i}\psi(x_i,x_j)$

Energy Function\
$E(x,y) = \sum_{i}\underbrace{\phi(x_i,y_i)}_\text{SNP} + \sum_{i,j}\underbrace{\phi(x_i,x_j)}_\text{PWP} $ 

Single-node Potential (Unary Potentials)
- Encode local information about the given pixel/patch

\
Pair-wise Potential (Binary Potentials)
- Encode neighborhood infromation


![](Segmentation/img/mrfstructure.png)





$ -logp(x,y) = - \sum_{i} \log \psi(x_i,y_i)$

Local Optima of Energy Function


## Segmentation Methods

### Graph Cut Method

One of the most effective methods in segmentation is the Graph Cut method. The main idea is to represent the image as a graph and find the cut between segments. In this section we explore converting EM problem to Graph problem. Main idea is to convert MRF into a source-sink graph. 

In order to understand better, first we need to make the terminology clear.

`Flow Network`: In graph theory, a flow network is a directed graph where each edge has a capacity and receives a flow. The vertices are called nodes, and edges are called arcs.

`Graph`: In Graph Theory, a Graph is a non-linear data structure consisting of vertices and edges. The vertices are sometimes also referred to as nodes and the edges are lines or arcs that connect any two nodes in the graph. More formally a Graph is composed of a set of vertices( V ) and a set of edges( E ). The graph is denoted by G(V, E).

`Cut`: A cut is a partition of the vertices of a graph into two disjoint subsets, such that the source node is in one set and the sink node is in the other. In a flow network, an `s–t` cut is a cut that requires the source and the sink to be in different subsets, and its cut-set only consists of edges going from the source's side to the sink's side. The capacity of an s–t cut is defined as the sum of the capacity of each edge in the cut-set.

### s-t MaxFlow MinCut Theorem

In optimization theory, the MaxFlow MinCut Theorem states that, in a flow network, the max amount of flow passing from the source to the sink is equal to the total weight of the edges in a minimum cut. This means that, if minimum cut, which is the smallest total weight of the edges, is removed, it would disconnect the source from the sink. In other words, the weight of a cut is the sum of the capacities of the edges that cross from one set to the other. A min-cut is a cut with the smallest total weight. In any flow network, the maximum amount of flow that can be pushed from the source to the sink is equal to the total weight of the edges in a minimum cut. The maximum flow that can be achieved is constrained by the smallest total capacity of the edges that, if removed, would disconnect the source from the sink. 

This theorem, at its core, states that the max flow one can achieve is directly related to the min bottleneck in the network that stops the flow.


Foreground: Source \
Background: Sink


Weights of a node $i$ $(x_i, y_i)$ with the node $j$ $(x_j, y_j)$ can be found by:

$$ \frac{e^{\frac{-||f(x_i, y_i)-f(x_j, y_j)||}{2\sigma^2}}}{\sqrt{(x_i-x_j)^2+(y_i-y_j)^2}}    $$ 


Here we will refer to the Matlab practice below

In [ ]:
import cv2
import networkx as nx
import numpy as np

'''

Graph *g;

For all pixels p 
     nodeID(p) = g->add_node();
     set_weights(nodeID(p),fgCost(p),bg(Cost(p)));
end

for all adjacent pixels p,q
    add_weights(nodeID(p),nodeID(q),cost(p,q);)

'''


img = cv2.imread('Image Processing\img\gaussian_original.jpg')

# Initialize a graph
g = nx.Graph()


def color_cost(p, q):
    color_diff = np.sqrt(np.sum((p - q)**2))
    return abs(sum(p) - sum(q))

# Functions to calculate foreground and background costs for a pixel
def fg_cost(p):

    return 0

def bg_cost(p):

    return 0

# Add nodes for all pixels and set weights
for pixel in img:
    node_id = g.add_node(pixel)
    fg_cost_val = fg_cost(pixel)
    bg_cost_val = bg_cost(pixel)
    g.nodes[node_id]['fg_cost'] = fg_cost_val
    g.nodes[node_id]['bg_cost'] = bg_cost_val

# Add edges between adjacent pixels and set weights
for p, q in nx.edges(g):
    cost_val = color_cost(p, q) 
    g.edges[p, q]['cost'] = cost_val

In [9]:
from collections import deque
def bfs(graph, start, search_value):
    visited = set()
    queue = deque([start])

    while queue:
        vertex = queue.popleft()
        if vertex == search_value:
            return True
        visited.add(vertex)
    
        for neighbour in graph[vertex]:
            if neighbour not in visited:
                queue.append(neighbour)
                visited.add(neighbour)
    return False

graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

start = "F"
search_value = "B"
res = bfs(graph, start, search_value)
print(f"element {search_value} : {res}")

element B : True


### Min Cut Bias and Normalized Cut

The min cut algorithm tends to produce small isolated segments which causes a bad partition. In order to adjust the weight of cut proportionally to the number of edges in the cut, we use the normalized cut solution which normalizes size of the segments and fixes the min cut bias.

$$ N_{cut}(A,B)=\frac{cut(A,B)}{assoc(A,V)} + \frac{cut(A,B)}{assoc(B,V)} $$

where
- $assoc(A,V)$ is the sum of weights of all edges that touch node $A$. 

Generalized eigenvalue problem is the approximate solution for minimizing the N-cut value. Let $W$ be the adjacency matrix of the graph. Let $D$ be the diagonal matrix with diagonal entries.

$$ D(i,j) = \sum_{i}w(i,j) $$

Then normalized cut cost can be written as

$$ \frac{y^T(D-w)y}{y^TDy} $$

where
- y is an indicator vector with the 1 in the $i^{th}$ position if the $i^{th}$ feature point belongs to A, negative constant otherwise.


### References for this Section

[1] Jun-DevpBlog https://medium.com/jun94-devpblog/cv-7-segmentation-as-energy-minimization-markov-random-fields-energy-formulation-graph-cut-670b9b3c82ee

[2] Statistical Techniques in Robotics, CMU https://www.cs.cmu.edu/~16831-f14/notes/F11/16831_lecture07_bneuman.pdf

[3] https://jwmi.github.io/ASM/Murphy%20chapter%2019.pdf

[4] https://en.wikipedia.org/wiki/Cut_(graph_theory)

[5] https://pub.ista.ac.at/~vnk/software.html

[6] https://julie-jiang.github.io/image-segmentation/

---

### Non-Binary Energy Problems

When dealing with non-binary energies, we are faced with the issue that multi-label problems are NP-hard when there are 3 or more labels. Approximation algorithms extend graph cuts to the multi label case and offer a solution:
- $\alpha$-Expansion
- $\alpha\beta$-Swap

They are no longer guaranteed to return the globally optimal result, however $\alpha$-Expansion has a guaranteed approximation quality (2-approx) and converges in a few iterations.

The key idea is to have multiclass segmentation problem, which means one node for every class, we pick one of the nodes -which is called "$\alpha$"- and assign it as the source, and then we pick one of the nodes from other labels (classes) and assign it sink.

Example: Stereo Vision and Depth Map
Depth maps contain information about the distance of objects from a specific perspective or reference point. Each pixel is assigned a value to represent the distance of that pixel from the reference point which creates a 3D representation of the scene for its RGB image or virtual scene. In each $\alpha$-expansion a given label $\alpha$ grabs space from other labels.

![](Segmentation/img/depthmapping.png)

![](Segmentation/img/depthmapgif.gif)

### References for this Section

[1] Looking Glass Factory https://lookingglassfactory.com/blog/depth-map

[2]

---